In [7]:
import os
import mne
import numpy as np
import cortex
import plotly.graph_objects as go
from scipy import stats
import plotly
from nilearn import datasets
import nibabel as nib
subjects_dir = mne.datasets.sample.data_path() / 'subjects'
labels = mne.read_labels_from_annot(
    'fsaverage', 'HCPMMP1', 'both', subjects_dir=subjects_dir)

Reading labels from parcellation...
   read 181 labels from /home/subbareddy/mne_data/MNE-sample-data/subjects/fsaverage/label/lh.HCPMMP1.annot
   read 181 labels from /home/subbareddy/mne_data/MNE-sample-data/subjects/fsaverage/label/rh.HCPMMP1.annot


In [8]:
fsaverage = datasets.fetch_surf_fsaverage("fsaverage6")
L_labels = nib.load('../afni-nosmooth/tpl-fsaverage6/tpl-fsaverage6_hemi-L_desc-MMP_dseg.label.gii')
R_labels = nib.load('../afni-nosmooth/tpl-fsaverage6/tpl-fsaverage6_hemi-R_desc-MMP_dseg.label.gii')

In [61]:
L_labels.labeltable.get_labels_as_dict()

{0: '???',
 1: 'L_V1_ROI',
 2: 'L_MST_ROI',
 3: 'L_V6_ROI',
 4: 'L_V2_ROI',
 5: 'L_V3_ROI',
 6: 'L_V4_ROI',
 7: 'L_V8_ROI',
 8: 'L_4_ROI',
 9: 'L_3b_ROI',
 10: 'L_FEF_ROI',
 11: 'L_PEF_ROI',
 12: 'L_55b_ROI',
 13: 'L_V3A_ROI',
 14: 'L_RSC_ROI',
 15: 'L_POS2_ROI',
 16: 'L_V7_ROI',
 17: 'L_IPS1_ROI',
 18: 'L_FFC_ROI',
 19: 'L_V3B_ROI',
 20: 'L_LO1_ROI',
 21: 'L_LO2_ROI',
 22: 'L_PIT_ROI',
 23: 'L_MT_ROI',
 24: 'L_A1_ROI',
 25: 'L_PSL_ROI',
 26: 'L_SFL_ROI',
 27: 'L_PCV_ROI',
 28: 'L_STV_ROI',
 29: 'L_7Pm_ROI',
 30: 'L_7m_ROI',
 31: 'L_POS1_ROI',
 32: 'L_23d_ROI',
 33: 'L_v23ab_ROI',
 34: 'L_d23ab_ROI',
 35: 'L_31pv_ROI',
 36: 'L_5m_ROI',
 37: 'L_5mv_ROI',
 38: 'L_23c_ROI',
 39: 'L_5L_ROI',
 40: 'L_24dd_ROI',
 41: 'L_24dv_ROI',
 42: 'L_7AL_ROI',
 43: 'L_SCEF_ROI',
 44: 'L_6ma_ROI',
 45: 'L_7Am_ROI',
 46: 'L_7PL_ROI',
 47: 'L_7PC_ROI',
 48: 'L_LIPv_ROI',
 49: 'L_VIP_ROI',
 50: 'L_MIP_ROI',
 51: 'L_1_ROI',
 52: 'L_2_ROI',
 53: 'L_3a_ROI',
 54: 'L_6d_ROI',
 55: 'L_6mp_ROI',
 56: 'L_6v_ROI',


In [9]:
language_rois = ['AG', 'ATL', 'PTL', 'IFG', 'MFG', 'IFGOrb', 'PCC', 'dmPFC']
AG_ROIs = ['PFm','PGs','PGi','TPOJ2','TPOJ3']
AG_indices = [149, 150, 151, 140, 141]

ATL_indices = [123, 131, 132, 134, 172, 128, 176]
ATL_regions = ['STGa','TGd','TE1a','TE2a','TGv', 'STSda', 'STSva']

ptl_rois = ['STSdp','A5','TPOJ1', 'PSL','STV', 'A4', 'STSvp']
ptl_indices = [129,125,139,25,28, 175, 130]

ifg = ['44', '45', 'IFJa', 'IFSp']
ifg_indices = [74, 75,  79, 81]

mfg = ['55b']
mfg_indices = [12]

ifgorb = ['a47r', 'p47r', 'a9-46v' ]
ifgorb_indices = [77, 171, 85]

pc = ['31', 'PCV', '7m', '23', 'RSC']
pc_indices = [161, 162, 27, 30, 32, 33, 34, 14]

dmpfc = [ '9m', '10d', 'd32']
dmpfc_indices = [ 69, 62, 72]

early_visual_names = ['V1','V2','V3']
early_visual_indices = [1,4,5]

dorsal_visual_names = ['V3A', 'V3B', 'V6','V6A', 'V7', 'IPS1']
dorsal_visual_indices = [13, 19, 3,152,16,17]

middle_temporal_names = ['MT', 'MST', 'LO1', 'LO2', 'FST', 'V3CD']
middle_temporal_indices = [23, 20,21, 157, 158]

language_indices = [AG_indices, ATL_indices, ptl_indices, ifg_indices, mfg_indices, ifgorb_indices, pc_indices, dmpfc_indices, 
                    early_visual_indices, dorsal_visual_indices, middle_temporal_indices]

In [10]:
models = ['video','audio','videomae']

In [58]:
normalized_scores = {}
subjects = ['1','3','5']
for eachmodel in models:
    normalized_scores[eachmodel] = []
    for eachsub in subjects:
        temp1 = []
        ceiling_voxcorrs = np.load('./movie10_predictions_results/noise_ceiling/subject_'+str(eachsub)+'_kernel_ridge.npy')
        model_voxcorrs = np.load('imagebind_both/'+str(eachsub)+'/'+eachmodel+'/all/'+'layer_0.npy')
        significant_indices = np.where(np.nan_to_num(ceiling_voxcorrs)<0.00)[0]
        model_voxcorrs[significant_indices] = 0
        normalize_across_voxels = model_voxcorrs/ceiling_voxcorrs
        normalize_across_voxels[np.where(normalize_across_voxels<0)[0]] = 0
        for eachroi in language_indices:
            temp = []
            for subroi in eachroi:
                lhindices = np.where(L_labels.darrays[0].data==subroi)[0]
                rhindices = np.where(R_labels.darrays[0].data==subroi)[0]
                lh_mean = normalize_across_voxels[0:40962][lhindices]
                rh_mean = normalize_across_voxels[40962:][rhindices]
                lh_rh = np.concatenate([lh_mean,rh_mean],axis=0)
                temp.append(np.mean(lh_rh))
            temp1.append(np.mean(temp))
        normalized_scores[eachmodel].append(np.array(temp1))
normalized_scores = np.array(list(normalized_scores.values()))
print(normalized_scores.shape)

(3, 3, 11)


In [107]:
np.mean(normalized_scores[0],axis=0)

array([0.29710758, 0.16357883, 0.19230221, 0.1471695 , 0.25732817,
       0.19068934, 0.22997956, 0.16116393])

In [108]:
np.mean(normalized_scores[1],axis=0)

array([0.14522962, 0.11744028, 0.29904012, 0.13491508, 0.18004535,
       0.24355203, 0.15333279, 0.19360927])

In [114]:
np.mean(normalized_scores[2],axis=0)

array([0.29711284, 0.16356988, 0.19228869, 0.14716659, 0.25732542,
       0.19067587, 0.22995659, 0.16115436])

In [115]:
normalized_scores = {}
subjects = ['1','3','5']
for eachmodel in models:
    normalized_scores[eachmodel] = []
    for eachsub in subjects:
        temp1 = []
        model_voxcorrs = np.load('imagebind_both/'+str(eachsub)+'/'+eachmodel+'/all/'+'layer_0.npy')
        model_voxcorrs[np.where(model_voxcorrs<0.0)[0]]=0.0
        for eachroi in language_indices:
            temp = []
            for subroi in eachroi:
                lhindices = np.where(L_labels.darrays[0].data==subroi)[0]
                rhindices = np.where(R_labels.darrays[0].data==subroi)[0]
                lh_mean = model_voxcorrs[0:40962][lhindices]
                rh_mean = model_voxcorrs[40962:][rhindices]
                lh_rh = np.concatenate([lh_mean,rh_mean],axis=0)
                temp.append(np.mean(lh_rh))
            temp1.append(np.mean(temp))
        normalized_scores[eachmodel].append(np.array(temp1))
normalized_scores = np.array(list(normalized_scores.values()))
print(normalized_scores.shape)

(3, 3, 8)


In [12]:
normalized_scores_bourn = {}
subjects = ['1','3','5']
for eachmodel in models:
    normalized_scores_bourn[eachmodel] = []
    for eachsub in subjects:
        temp1 = []
        ceiling_voxcorrs = np.load('./movie10_predictions_results_bourn/noise_ceiling/subject_'+str(eachsub)+'_kernel_ridge.npy')
        model_voxcorrs = np.load('imagebind_both/'+str(eachsub)+'/'+eachmodel+'/bourn/'+'layer_0.npy')
        significant_indices = np.where(np.nan_to_num(ceiling_voxcorrs)<0.05)[0]
        model_voxcorrs[significant_indices] = np.nan
        normalize_across_voxels = model_voxcorrs/ceiling_voxcorrs
        normalize_across_voxels[np.where(normalize_across_voxels<0)[0]] = 0
        normalize_across_voxels[np.where(normalize_across_voxels>1)[0]] = 1
        for eachroi in language_indices:
            temp = []
            for subroi in eachroi:
                lhindices = np.where(L_labels.darrays[0].data==subroi)[0]
                rhindices = np.where(R_labels.darrays[0].data==subroi)[0]
                lh_mean = normalize_across_voxels[0:40962][lhindices]
                rh_mean = normalize_across_voxels[40962:][rhindices]
                lh_rh = np.concatenate([lh_mean,rh_mean],axis=0)
                temp.append(np.nanmean(lh_rh))
            temp1.append(np.nanmean(temp))
        normalized_scores_bourn[eachmodel].append(np.array(temp1))
normalized_scores_bourn = np.array(list(normalized_scores_bourn.values()))
print(normalized_scores_bourn.shape)

(3, 3, 11)


In [15]:
normalized_scores_wolf = {}
subjects = ['1','3','5']
for eachmodel in models:
    normalized_scores_wolf[eachmodel] = []
    for eachsub in subjects:
        temp1 = []
        ceiling_voxcorrs = np.load('./movie10_predictions_results_bourn/noise_ceiling/subject_'+str(eachsub)+'_kernel_ridge.npy')
        model_voxcorrs = np.load('imagebind_both/'+str(eachsub)+'/'+eachmodel+'/wolf/'+'layer_0.npy')
        significant_indices = np.where(np.nan_to_num(ceiling_voxcorrs)<0.05)[0]
        model_voxcorrs[significant_indices] = np.nan
        normalize_across_voxels = model_voxcorrs/ceiling_voxcorrs
        normalize_across_voxels[np.where(normalize_across_voxels<0)[0]] = 0
        normalize_across_voxels[np.where(normalize_across_voxels>1)[0]] = 1
        for eachroi in language_indices:
            temp = []
            for subroi in eachroi:
                lhindices = np.where(L_labels.darrays[0].data==subroi)[0]
                rhindices = np.where(R_labels.darrays[0].data==subroi)[0]
                lh_mean = normalize_across_voxels[0:40962][lhindices]
                rh_mean = normalize_across_voxels[40962:][rhindices]
                lh_rh = np.concatenate([lh_mean,rh_mean],axis=0)
                temp.append(np.nanmean(lh_rh))
            temp1.append(np.nanmean(temp))
        normalized_scores_wolf[eachmodel].append(np.array(temp1))
normalized_scores_wolf = np.array(list(normalized_scores_wolf.values()))
print(normalized_scores_wolf.shape)

(3, 3, 11)


In [13]:
normalized_scores_bourn[0]

array([[0.31424824, 0.19629156, 0.16390726, 0.12285774, 0.3073514 ,
        0.1647061 , 0.08087434, 0.43956268, 0.3803697 , 0.14922871,
        0.57972888],
       [0.31793337, 0.17509475, 0.06614564, 0.02298167, 0.03990355,
        0.0874477 , 0.24685762, 0.11714988, 0.19712788, 0.14304823,
        0.31776915],
       [0.33410292, 0.13980965, 0.22864804, 0.19280425, 0.45703145,
        0.09759927, 0.40070425, 0.04452225, 0.26633159, 0.28777964,
        0.32710353]])

In [16]:
normalized_scores_wolf[0]

array([[0.24057275, 0.14509613, 0.11971373, 0.12561706, 0.19992606,
        0.10839644, 0.06325616, 0.31169042, 0.25537646, 0.08837715,
        0.38915717],
       [0.20865011, 0.21234124, 0.07135   , 0.01689403, 0.1026605 ,
        0.13341239, 0.17126921, 0.12023694, 0.09623271, 0.10042285,
        0.183526  ],
       [0.19434513, 0.08249828, 0.09925869, 0.08149055, 0.28564044,
        0.10631782, 0.25448318, 0.08025962, 0.19344083, 0.19363405,
        0.21666353]])

In [14]:
np.mean(normalized_scores,axis=1)

array([[0.29710758, 0.16357883, 0.19230221, 0.1471695 , 0.25732817,
        0.19068934, 0.22997956, 0.16116393, 0.23562543, 0.20203714,
        0.38705762],
       [0.14522962, 0.11744028, 0.29904012, 0.13491508, 0.18004535,
        0.24355203, 0.15333279, 0.19360927, 0.17198498, 0.11503877,
        0.14248944],
       [0.26149061, 0.19986592, 0.18126771, 0.2544251 , 0.2544907 ,
        0.23552172, 0.28961344, 0.31152686, 0.25968249, 0.27839431,
        0.26350851]])

In [56]:
ceiling_scores = []
subjects = ['1','3','5']
for eachsub in subjects:
    ceiling_voxcorrs = np.load('./movie10_predictions_results/noise_ceiling/subject_'+str(eachsub)+'_kernel_ridge.npy')
    ceiling_voxcorrs[np.where(ceiling_voxcorrs<0.01)[0]] = 0
    #ceiling_scores.append(np.mean(ceiling_voxcorrs[np.where(ceiling_voxcorrs>0.0)[0]]))
    temp1 = []
    for eachroi in language_indices:
        temp = []
        for subroi in eachroi:
            lhindices = np.where(L_labels.darrays[0].data==subroi)[0]
            rhindices = np.where(R_labels.darrays[0].data==subroi)[0]
            lh_mean = ceiling_voxcorrs[0:40962][lhindices]
            rh_mean = ceiling_voxcorrs[40962:][rhindices]
            lh_rh = np.concatenate([lh_mean,rh_mean],axis=0)
            temp.append(np.mean(lh_rh))
        temp1.append(np.mean(temp))
    ceiling_scores.append(np.array(temp1))
ceiling_scores = np.array(ceiling_scores)
print(ceiling_scores)

[[0.11319428 0.09463792 0.13297797 0.12728798 0.04338337 0.10217253
  0.08629681 0.04480407 0.11525301 0.08678962 0.09167857]
 [0.09238356 0.06505742 0.11695341 0.10241339 0.19915225 0.08914682
  0.07866969 0.0870273  0.147847   0.10735229 0.14857475]
 [0.09058857 0.12569542 0.09961319 0.17853056 0.16096229 0.07399929
  0.06811681 0.1212945  0.12305787 0.07826408 0.14902535]]


In [57]:
ceiling_scores[2]

array([0.09058857, 0.12569542, 0.09961319, 0.17853056, 0.16096229,
       0.07399929, 0.06811681, 0.1212945 , 0.12305787, 0.07826408,
       0.14902535])

In [59]:
np.mean(normalized_scores,axis=1)

array([[2.74755118, 0.41799843, 0.58199753, 0.21896902, 0.6205619 ,
        2.23426781, 1.55465249, 0.30296217, 0.5806428 , 0.27205877,
        2.10678133],
       [0.50084171, 0.19040453, 0.91078805, 0.27536582, 0.83694681,
        1.08445684, 0.62836645, 0.30855439, 0.5045676 , 0.13468541,
        0.34747441],
       [0.8744242 , 0.61290592, 0.76472255, 0.80015655, 0.44520572,
        1.20185373, 1.27608372, 0.56036837, 0.76064738, 0.43496293,
        2.11853701]])